##### from cad.calc.geo import Geo, geotools
from cad.calc.didgmo import didgmo_high_res
from cad.calc.visualization import DidgeVisualizer

geo=[[0,32], [2300, 80]]
geo=Geo(geo=geo)
fft=didgmo_high_res(geo)

DidgeVisualizer.vis_didge(geo)

In [20]:
from cad.calc.parameters import BubbleParameters
import pandas as pd

ffts=[]
ffts.append(fft.peaks)

n=5
for i in range(n):
    geo2=BubbleParameters(geo.copy(), 0.5, 1+i/n, 200).make_geo()
    fft2=didgmo_high_res(geo2)
    ffts.append(fft2.peaks)
    #DidgeVisualizer.vis_didge(geo2)

df={}
for i in range(len(ffts)):
    df[i]=ffts[i]["freq"]

#df[0]
df=pd.DataFrame(df)
df
#fft.peaks["note-name"] + " " + fft.peaks["cent-diff"].apply(lambda x : str(round(x,2)))

,0,1,2,3,4,5
0,45.55,52.55,52.25,51.75,51.20,50.50
1,103.50,114.00,114.10,113.95,113.65,113.25
2,168.80,188.50,187.30,185.80,184.00,182.05
3,237.85,256.80,257.20,257.20,256.90,256.45
4,310.00,333.00,330.00,328.00,326.00,324.00
5,382.00,403.00,405.00,406.00,406.00,406.00
6,455.00,476.00,473.00,471.00,468.00,466.00
7,528.00,548.00,551.00,554.00,556.00,557.00
8,602.00,617.00,613.00,608.00,704.00,707.00
9,675.00,691.00,697.00,701.00,849.00,853.00


,0,1,2,3,4,5
0,45.55,52.15,51.75,47.80,47.85,47.60
1,103.50,117.80,113.95,117.70,116.85,108.05
2,168.80,188.45,185.80,185.35,187.05,181.35
3,237.85,260.60,257.20,247.50,243.70,254.25
4,310.00,334.00,328.00,331.00,316.00,314.00
5,382.00,407.00,406.00,398.00,398.00,379.00
6,455.00,481.00,471.00,460.00,479.00,457.00
7,528.00,554.00,554.00,545.00,531.00,539.00
8,602.00,628.00,608.00,614.00,601.00,618.00
9,675.00,701.00,701.00,674.00,685.00,675.00
